**Ch2 sequence analysis**

* in preliminary CH1 analyses, I did some sequence trimming in conda locally. It is more streamlined to do trimming in qiime
* manifest file directs qiime to find raw files in RIS storage

* activate Qiime2

In [ ]:
## in WashU RIS cluster: see RIS document for context but examples below)

bsub -Is -G compute-rachel.penczykowski -q general-interactive -R 'rusage[mem=16GB]' -a 'docker(quay.io/qiime2/core:2023.2)' /bin/bash
bsub -Is -G compute-rachel.penczykowski -q general-interactive -R 'rusage[mem=32GB]' -a 'docker(quay.io/qiime2/core:2023.2)' /bin/bash
bsub -Is -G compute-rachel.penczykowski -q general-interactive -R 'rusage[mem=128GB]' -a 'docker(quay.io/qiime2/core:2023.2)' /bin/bash

* change directory to files location
* my manifest file directs qiime to find raw files in RIS storage

In [ ]:
cd /storage1/fs1/rachel.penczykowski/Active/ptanford/CH2/ #put manifest file in rs1 folder

qiime tools import \
--type 'SampleData[PairedEndSequencesWithQuality]' \
--input-path CH2_manifest.txt \
--output-path raw_seqs.qza \
--input-format PairedEndFastqManifestPhred33V2 

## demux summarize (notes on why we do this to hopefully follow but at a minimum sanity check of import success)
## I believe the main point of this is to have sequence statistics as data quality etc. checks visualizable at
## view.qiime2.org

qiime demux summarize \
--i-data raw_seqs.qza \
--o-visualization raw_seqs.qzv 

**REMOVING PRIMERS + adapters and DISPOSING OF NON-PRIMER READS **
**I think I can remove primer + adapter together. For CH1 it was important to separate fungi and bacteria**

* fITS7 GTGAATCATCGAATCTTTG (Ihrmark et al. 2012) 
* ITS4 TCCTCCGCTTATTGATATGC (White et al. 1990)

* Forward overhang: TCGTCGGCAGCGTCAGATGTGTATAAGAGACAG
* Reverse overhang: GTCTCGTGGGCTCGGAGATGTGTATAAGAGACAG

* adfITS7 TCGTCGGCAGCGTCAGATGTGTATAAGAGACAGGTGAATCATCGAATCTTTG
* adITS4 GTCTCGTGGGCTCGGAGATGTGTATAAGAGACAGTCCTCCGCTTATTGATATGC

In [ ]:
## remove primers (adapter overhang + ITS primer seq) with cutadapt

qiime cutadapt trim-paired \
--i-demultiplexed-sequences raw_seqs.qza \
--p-front-f TCGTCGGCAGCGTCAGATGTGTATAAGAGACAGGTGAATCATCGAATCTTTG \
--p-front-r GTCTCGTGGGCTCGGAGATGTGTATAAGAGACAGTCCTCCGCTTATTGATATGC \
--p-match-read-wildcards \
--p-match-adapter-wildcards \
--verbose \
--o-trimmed-sequences raw_seqs_trim1.qza

## assess files post trim
qiime demux summarize \
--i-data raw_seqs_trim1.qza \
--o-visualization trimmed_demuxsum.qzv 

**Run dada2 for denoising**

In [ ]:
## paired maxEE 2,2
qiime dada2 denoise-paired \
--i-demultiplexed-seqs raw_seqs_trim1.qza \
--p-trunc-len-f 0 \
--p-trunc-len-r 10 \
--p-max-ee-f 2 \
--p-max-ee-r 2 \
--o-table dada2_table_22.qza \
--o-denoising-stats maxEE2-2/dada2_stats_22.qza \
--o-representative-sequences maxEE2-2/RepSeqs22.qza \
--verbose

## paired no maxEE - no reads merge. will have to go with single
qiime dada2 denoise-paired \
--i-demultiplexed-seqs raw_seqs_trim1.qza \
--p-trunc-len-f 0 \
--p-trunc-len-r 10 \
--o-table dada2_table_noEE.qza \
--o-denoising-stats dada2_stats_noEE.qza \
--o-representative-sequences RepSeqs_noEE.qza \
--verbose



In [ ]:
## single maxEE 2 (so only forward reads)
qiime dada2 denoise-single \
--i-demultiplexed-seqs raw_seqs_trim1.qza \
--p-max-ee 2 \
--p-trunc-len 0 \
--o-table dada2_table_2.qza \
--o-denoising-stats dada2_stats_2.qza \
--o-representative-sequences RepSeqs_2.qza \
--verbose


In [ ]:
qiime feature-table summarize \
  --i-table dada2_table_2.qza \
  --o-visualization feature_table_2.qzv \
  --m-sample-metadata-file CH2_metadata.txt
  
qiime feature-table tabulate-seqs \
--i-data RepSeqs_2.qza \
--o-visualization RepSeqs_2.qzv

**Classify taxa with unite database classifier**

In [ ]:
## copying code I used to make prior fungal classifier. Will use it again to classify these taxa but I am not running this anew
## the below only ran on the cluster with 128GB RAM. 64 was not enough
qiime feature-classifier fit-classifier-naive-bayes \
 --i-reference-reads unite-refs-ver9_99_25.07.2023.qza \
 --i-reference-taxonomy unite-taxonomy-ver9_99_25.07.2023.qza \
 --o-classifier unite-2023-classifier.qza


## classify my samples
## this doesn't run on 64GB
qiime feature-classifier classify-sklearn \
  --i-classifier unite-2023-classifier.qza \
  --i-reads RepSeqs_2.qza \
  --o-classification Taxonomy_2.qza



**Sanity check - diversity plots (prior to phylogenetic IDs) in qiime2**

In [ ]:
qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences RepSeqs_2.qza \
  --o-alignment testAlignedRepSeqs.qza \
  --o-masked-alignment testMaskedRepSeqs.qza \
  --o-tree testUnrooted-tree.qza \
  --o-rooted-tree testRooted-tree.qza

qiime diversity core-metrics-phylogenetic \
  --i-phylogeny testRooted-tree.qza \
  --i-table dada2_table_2.qza \
  --p-sampling-depth 10000 \
  --m-metadata-file CH2_metadata.txt \
  --output-dir core-metrics-results

## plots look overall good!! samples clustering by soil type. much more spread 
## of live samples with jaccard - unique fungal spp in live pots


Troubleshooting qza to phyloseq issues - possibly due to my sample names being numbers. Per this:
https://forum.qiime2.org/t/change-sample-ids-after-running-dada2/3918
I can use the feaature table group function to direct qiime to new sample IDs

"Luckily, I have just the trick for you! Take a look at feature-table group! That might sound weird at first, but, stick with me. What you can do is add a new column to your metadata file that contains the new sample IDs. Then, when you run the group command, point the --m-metadata-column to the column containing your new sample IDs. The --p-mode parameter don't matter (you can pick whatever you want) - because you are "grouping" the values into the same "groups", just with new names, none of the abundances will change. Et voila! Moving forward, just drop the old sample ID column from your metadata and replace it with the new sample ID column. :recycle:"

In [ ]:
feature table group \
--i-table \
--p-axis [feature|sample] \
--m-metadata-file MULTIPLE PATH \
--m-metadata-column MetadataColumn[Categorical] \
--p-mode [median-ceiling|sum|mean-ceiling] \ 
--o-grouped-table ARTIFACT PATH FeatureTable[Frequency]

#p-mode is required but arbitrary

qiime feature-table group \
--i-table dada2_table_2.qza \
--p-axis sample \
--m-metadata-file CH2_metadata_newIDs.txt \
--m-metadata-column sample \
--p-mode sum \
--o-grouped-table dada2_table_2_newIDs


The above ran! I then had to:
* Save a new keemi-approved metadata sheet that replaced the prior sample-id names (101, 102...., under the #q2:types column) with sample (Sample101, Sample102...), deleting the sample-id column entirely. The "CH2_metadata_newIDs" file has both sample types in order to run the grouping code and output a new feature table with the new ids. "Ch2_metadata_newIDs2" (note the 2 at the end) is the new metadata file associated with the new dada2 table, for use for importing into R with qza to phyloseq :)